## 형태소분석_명사추출

- 형태소 분석을 먼저하고
- 거기서 띄어쓰기없이 쭉 이어져 있는 부분들에 대하여 인위적으로 띄어쓰기 넣어주기

In [1]:
with open('data/cleaned_playlist_pje.txt') as f:
    playlist = f.read().splitlines()
    
with open('data/preanal.txt') as f:
    data = f.read().splitlines()

In [2]:
len(playlist)

115071

In [3]:
playlist

['여행같은 음악',
 '요즘 너 말야',
 '편하게  잔잔하게 들을 수 있는 곡  ',
 '크리스마스 분위기에 흠뻑 취하고 싶을때',
 '추억의 노래  ',
 '2017 Pop Trend',
 '짝사랑  고백  사랑  이별  슬픔   감성을 자극하는곡들 ',
 '멍청이   내맘도 몰라 ',
 'DANCING IN THE MOON LIGHT  01',
 ' 록 메탈  Written by 이일우',
 '걸그룹 땐쓰쏭 ',
 '노래로 의지를 불태우자   1일1다짐  st용 프로필뮤직',
 '지친 너를 위로해줄 제목편지 준비해봤어',
 '트렌디하고 그루브한 힙합 알앤비 MUSIC',
 'Autumn in Jazz',
 '걍게임할때듣는음악',
 '나만 알고싶은 노래들',
 '축가듀엣',
 '이렇게 비 내리는 날이면  너도 내 생각 할까  ',
 '올림픽 스밍 목록',
 '조용히 맥주 한잔 할때 새벽감성',
 'ᴡʜɪᴛᴇ   ʀᴏᴍᴀɴᴛɪᴄ ᴊᴀᴢᴢ ᴀᴛ ᴀ ᴄᴀғᴇ',
 '추억의 명화를 만나는 시간  영화 OST 모음 ',
 '  기분좋은 여름날  ',
 '지친 하루 끝  힐링이 필요한 당신에게 추천하는 인디곡',
 '새벽 찬바람이 침대위를 감쌀때   국외 ',
 '기분좋은 햇살같은 숨겨진 명곡들  ',
 '사랑스러운 남녀듀엣',
 '즐겨듣는소래',
 '닥힙추 8월 Playlist',
 '거슈윈에서 스타워즈까지  미국 대표 클래식',
 '겨울이 기다리며 미리 들어요   ',
 '아침이 상쾌해지는 빅밴드의 스윙스윙  ',
 '여행가는 길 이런음악과 함께 고고씽   ',
 '믿고듣는 하우스 일렉트로니카 POP  2',
 '프로포즈 음악',
 '보아일본곡',
 '안다르 레깅스 입고 운동 시작하자 ',
 '발라드로 감성을 발라드림 슬픔주의 ',
 '크리스마스를 책임질 캐롤 모음',
 '아무것도 위로 되지 못할 때 위로가 되어준 음악',
 '트렌디한 Cafe Pop Music',
 '8 90년대 우리나라 라디오에서 흘러나오던 명곡들 ',
 '영화  Jazz를 만나다 294',

In [4]:
for i in range(0,len(playlist)):
    if len(playlist[i])==0:
        print('yes')

### 명사추출

In [5]:
import re
import itertools
nouns = []

# 모든 NNP/NNG/NP추출
for i in range(0,len(data)):
   tmp = list([item for item in data[i].replace('\t'," ").split(" ") if re.search('(NNP$)|(NNG$)|(NP$)', item) and (len(item)>=6)])
   if len(tmp)!=0:
       nouns.append(tmp)

#nouns unlist 진행
nouns = list(itertools.chain(*nouns))
nouns_final = list(set([re.sub("/NNG|/NNP|/NP","",item) for item in nouns])) #최종 preanal에 있는 unique 명사

## Khaiii ㄱㄱ

In [6]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

**이거를 병렬처리**

In [ ]:
morphs = []
tmp_list = []

for i in range(0,len(playlist)):
    if playlist[i].find(" ")==-1: #띄어쓰기 없는 것만 수동띄어쓰기
        for j in range(0,len(nouns_final)):
            playlist[i] = re.sub("(\w*)("+nouns_final[j]+")(\w+)|(\w+)("+nouns_final[j]+")(\s+)|(\w+)("+nouns_final[j]+")(\w+)",r"\1 \2 \3",playlist[i])
        print(playlist[i])
    if playlist[i].isspace():
        continue
    for word in api.analyze(playlist[i]):
        for morph in word.morphs:
            # 실질형태소 가져오기
            if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ']:
                tmp_list.append(morph.lex)
    morphs.append(tmp_list)
    tmp_list = []
    print(i)

### 병렬처리

### Pool

In [50]:
from multiprocessing import Pool
import time

def do_work(i):
    tmp_list = []

    if playlist[i].find(" ")==-1: #띄어쓰기 없는 것만 수동띄어쓰기
        for j in range(0,len(nouns_final)):
            playlist[i] = re.sub("(\w*)("+nouns_final[j]+")(\w+)|(\w+)("+nouns_final[j]+")(\s+)|(\w+)("+nouns_final[j]+")(\w+)",r"\1 \2 \3",playlist[i])
        print(playlist[i])
#     if playlist[i].isspace():
#         continue
    for word in api.analyze(playlist[i]):
        for morph in word.morphs:
            # 실질형태소 가져오기
            if morph.tag in ['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ']:
                tmp_list.append(morph.lex)
    print(i)
    return tmp_list
    


if __name__ == '__main__':
    start_time = time.time()
    with Pool(processes=4) as p:
        result = p.map(do_work, range(0,len(playlist)))
#         result = [r for r in res]
    print("--- %s seconds ---" % (time.time()-start_time))

0143847192
21576

1
143857193
21577

2
719414386


3719514387


7196414388


7197514389


143906

143917

143928

14393
9
10
11
12
13
14
피겨스케이팅 겨울 이네요

719821578
 수정 이앨범
7199
21579
14394

72002158014395


720121581
14396
7202
28768
143977203


28769143987204


14399287707205


14400287717206
걍 게임 할때듣는음악

14401
287727207
15

14402
720828773
16

14403
720928774


144047210

144057211

144067212

144077213

144087214

144097215

144107216

7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
축가듀엣
17
18
19
20
21
22 크리스 마스127

2328775

2428776

25
26한경일

2714411

35960
35961
35962
35963
35964
35965
35966
35967
35968
35969
35970
35971
35972
35973
35974
호림뮤직
7243
7244
7245
7246
7247
7248
7249
즐겨듣는소래
28 크리스 마스149

2928777

3028778

3128779

3228780

3328781

3428782

35
 라인 댄스
35975
35976
35977
35978
   의 친구 입니다
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
72

Process ForkPoolWorker-45:
Process ForkPoolWorker-48:
Process ForkPoolWorker-46:
Process ForkPoolWorker-47:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/p

KeyboardInterrupt: 

In [48]:
result

[['여행', '같', '음악'],
 ['요즘', '너', '말'],
 ['편하', '잔잔', '듣', '있', '곡'],
 ['크리스마스', '분위기', '흠뻑', '취하', '때'],
 ['추억', '노래'],
 [],
 ['짝사랑', '고백', '사랑', '이별', '슬픔', '감성', '자극', '곡'],
 ['멍청', '나', '맘', '모르'],
 [],
 ['록', '메', '탈', '이일우'],
 ['걸그룹', '때', '쓰쏭'],
 ['노래', '의지', '불태우', '짐', '프로', '필뮤직'],
 ['지치', '너', '위로', '제목', '편지', '준비'],
 ['트렌디', '그루브', '힙합', '알앤비'],
 [],
 ['걍', '게임', '하', '때', '듣', '음악'],
 ['나', '알', '노래'],
 ['축가듀엣'],
 ['이렇', '비', '내리', '날', '너', '나', '생각', '하'],
 ['올림픽', '스밍', '목록']]

In [49]:
len(result)

20

## 여기는 띄어쓰기없는애들 그냥 봐봤다!

In [216]:
import copy
a = copy.deepcopy(data)

In [217]:
no_list = []
for i in range(0,len(a)):
    if a[i].find(" ")==-1:
        no_list.append(a[i])

In [218]:
no_list

['걍게임할때듣는음악',
 '축가듀엣',
 '즐겨듣는소래',
 '보아일본곡',
 '군산대쿠바클럽노래',
 '5617',
 '매일듣는노래',
 'JAZZYMOOD',
 '쇼미더머니8노래모음',
 '가을저녁',
 '크리스마스9',
 'lsos',
 '디제잉휘',
 '3호선버터플라이',
 '멜로하군요',
 '여름방학',
 '테크노음악',
 '2019',
 '감성재즈',
 '듣는편지',
 '감성적인겨울발라드',
 'Playlist3',
 '콜라보레이션',
 '설레는노래',
 '감성폭발',
 'Addicted',
 'joony',
 '노래모음',
 '조용필',
 '들어봐',
 '운동할때',
 'DJ열이',
 '추천곡곡',
 'lol노래',
 '신나는노래',
 '부활',
 '멍때리자',
 '감성발라드',
 'Festival',
 '감성자극음악모음',
 '울고웃는잔잔한노래',
 '응팔ost모음집',
 '리드미컬2',
 'holic',
 '요요마곡',
 '완전감동음악',
 '달달하게',
 'Nostalgia',
 'Relaxed',
 'FEMINISM',
 '겁클파이팅',
 '디제이플레이',
 'pop',
 '락발라드',
 '이갈룡폴더',
 '0901',
 '신나는크리스마스',
 'miru0914',
 '락초보이야기',
 '신콘서트',
 'WAVYVAW',
 '비틀즈노래모음집',
 'chill',
 'pppp',
 'xxxxx',
 '수정이앨범',
 '크리스마스156',
 '보사어쿠스틱',
 'HIT4864',
 '여름이야아아아아아',
 '플레이리스트',
 '부활',
 '추억의ost',
 '테스트7',
 '봄봄봄봄',
 '밍키노래',
 '들어보시오',
 '클래씩이',
 'Melancolia',
 '일렉트로니카',
 '버스안퇴근길창가느낌',
 '와이파이홈트용',
 'soulful',
 '수정이앨범',
 '조용한팝',
 '자기전에',
 '차량용',
 '달달한밤빈티지팝송',
 '잊고살기엔아까운아련한곡들',
 '지식채널e',
 '방송할때키는음악2',
 '이젠잊을래',
 '나만알고싶

### Khaiii ㄱㄱ

In [219]:
from khaiii import KhaiiiApi
api = KhaiiiApi()

In [220]:
morphs = []
for i in range(0,len(no_list)):
    for word in api.analyze(no_list[i]):
        print(word)
        morphs.append(word)

걍게임할때듣는음악	걍/MAG + 게/MAG + 임/NNG + 하/XSA + ㄹ/ETM + 때듣/VV + 는/ETM + 음악/NNG
축가듀엣	축가듀엣/NNG
즐겨듣는소래	즐기/VV + 어/EC + 듣/VV + 는/ETM + 소래/NNG
보아일본곡	보/VV + 아일본곡/NNP
군산대쿠바클럽노래	군산대쿠/NNP + 바클럽노래/NNG
5617	5617/SN
매일듣는노래	매일듣/MAG + 는/ETM + 노/MAG + 래/NNG
JAZZYMOOD	JAZZYMOOD/SL
쇼미더머니8노래모음	쇼미더머니/NNG + 8/SN + 노래모음/NNG
가을저녁	가을저녁/NNG
크리스마스9	크/NNG + 리스마/NNP + 스/NNG + 9/SN
lsos	lsos/SL
디제잉휘	디제잉휘/NNP
3호선버터플라이	3/SN + 호/NNB + 선버터플/NNG + 라/NNP + 이/NNG
멜로하군요	멜로/NNG + 하군/NNG + 요/EC
여름방학	여름/NNG + 방학/NNG
테크노음악	테크노음악/NNG
2019	2019/SN
감성재즈	감/NNG + 성재즈/NNP
듣는편지	듣/VV + 는편/EC + 지/NNG
감성적인겨울발라드	감성/NNG + 적/XSN + 이/VCP + ㄴ/ETM + 겨울발라/NNG + 드/NNP
Playlist3	Playlist/SL + 3/SN
콜라보레이션	콜라보레이션/NNG
설레는노래	설/VV + 레/NNG + 는/JX + 노래/NNG
감성폭발	감성/NNG + 폭발/NNG
Addicted	Addicted/SL
joony	joony/SL
노래모음	노래/NNG + 모음/NNG
조용필	조용필/NNP
들어봐	듣어/VV + 보/VX + 아/EC
운동할때	운동/NNG + 하/XSV + ㄹ/ETM + 때/NNG
DJ열이	DJ/SL + 열/NNG + 이/JKS
추천곡곡	추천/NNG + 곡곡/NNG
lol노래	lol/SL + 노래/NNG
신나는노래	신/XPN + 나는노래/NNG
부활	부활/NNG
멍때리자	멍때리/NNG + 자/EC
감성발라드	감/NNG + 성발라드/NNP
Festival	F

In [77]:
print(playlist.index(""))

52495


In [221]:
len(no_list)

6589

In [222]:
len(morphs)

6589